<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# PCA Lab: PCA Visualization and Horn's Parallel Analysis

_Author: Kiefer Katovich (SF)_

---

**Outline:**

- [Part I](#parti): Guided PCA example with the [heptathlon performance data set](./datasets/heptathlon.csv).
- [Part II](#partii): Try PCA yourself with the [wine quality data set](./datasets/wine_quality.csv).
- [Part III](#partiii): Use Horn's parallel analysis to select the number of components.

**In this lab, we will:**

- Practice cleaning data.
- Perform PCA and interpret the principal components.
- Validate PCA results using visualizations and intuition.
- Select the number of principal components using elbow plots and Horn's parallel analysis.

_Horn's parallel analysis_ is a way to determine how many components you should keep after performing PCA on your data. Essentially, it will tell you which of your components are likely noise and can therefore be discarded.


---

<a id="parti"></a>
# Part I: Heptathlon Data Set


### 1) Load packages and heptathlon data.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

plt.style.use('fivethirtyeight')

from ipywidgets import *
from IPython.display import display

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# The athlete names make a natural index and are non-numeric,
#   so there’s no need to exclude them when creating matrices.
hep = pd.read_csv('./datasets/heptathlon.csv', index_col=0)

hep.head(3)

---

### 2) Create a DataFrame that excludes `athlete` and `score`.

In [ ]:
# Double check the number of entries, the number of null values, and the proper data types.
hep.info()

In [ ]:
hep.head()

In [ ]:
# Note that in some events, a high number is good, whereas in others it’s bad.
# - Javelin is based on distance thrown (higher is better).
# - Shot is based on distance thrown (higher is better).
# - Long jump is based on distance jumped (higher is better).


# Problem: Running events are based on time taken (lower is better).
#   - So, we'll normalize to m/s (higher is better).

hep['hurdles'] = 110. / hep['hurdles']
hep['run200m'] = 200. / hep['run200m']
hep['run800m'] = 800. / hep['run800m']

hep.head()

---

### 3) Examine the correlation between the different events.

Plot a heat map if you want to visualize the correlation. What does the correlation matrix tell you?

In [ ]:
# A:

In [ ]:
# A:

**Note that all correlations are positive.** Why? 

Recall that we altered the data such that higher numbers always indicate a higher score and likely a better athlete overall.

---

### 4) Standardize the data.

In [ ]:
# Split into features and targets.
#   - You can control the order of the event columns by changing their positions here:

EVENT_NAMES = ['hurdles', 'run200m', 'run800m', 'highjump', 'shot', 'longjump', 'javelin']
TARGET_NAMES = ['score']

X = hep[EVENT_NAMES]
X.head()

In [ ]:
# A:

---

### 5) Fit a PCA on the standardized data using scikit-learn.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(Xn)

In [ ]:
pca.components_

In [ ]:
pc1_ev = pca.components_[0]

# A quick way of viewing, rather than looping through each feature:
pd.Series(pc1_ev, index=EVENT_NAMES)

In [ ]:
pc2_ev = pca.components_[1]

pd.Series(pc2_ev, index=EVENT_NAMES)

---

### 6) Create a DataFrame with the principal components.

Add back in the `athlete` and `score` columns from the original data.

In [ ]:
pca.components_[0]

In [ ]:
events.head(1)

In [ ]:
hep_pcs = pca.transform(Xn)
hep_pcs = pd.DataFrame(hep_pcs, 
                       columns=['PC'+str(i+1) for i in range(len(EVENT_NAMES))],
                       index=hep.index)
hep_pcs['score'] = hep['score']
hep_pcs.head()

In [ ]:
# Always make sure you verify that your numbers are correct as often as possible!
#   - At a minimum, let's ensure the athletes and scores still match up.
hep.head(2)

---

### 7) Plot the explained variance (ratio) of your components.

Explain what this chart tells you about your components.

In [ ]:
exp_var = pca.explained_variance_ratio_
exp_var

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(range(1, len(exp_var)+1), exp_var, lw=2)
ax.scatter(range(1, len(exp_var)+1), exp_var, s=120)
ax.set_title('Explained variance pct\n', fontsize=20)
ax.set_xlabel('Principal Component', fontsize=16)
ax.set_ylabel('Variance Explained (%)', fontsize=16)
plt.show()

---

### 8) Print out the weights/eigenvectors (`.components_` ) with their corresponding variables for PC1 and PC2.

Based on how the original variables are weighted to calculate the components, how would you describe PC1 and PC2?

In [ ]:
# Reminder of PC1 and PC2:

pd.DataFrame({'PC1': pc1_ev, 'PC2': pc2_ev},
             index=EVENT_NAMES)

PC1 describes athletes who are below the mean in all categories. We will describe them as "below average.”

PC2 describes athletes who are the best at javelin. We will describe them as "javelin.”

---

### 9) Plot PC1 versus PC2. Which athletes are notable on each component?

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
ax.scatter(hep_pcs.PC1.values, hep_pcs.PC2.values, s=0)

for i, txt in enumerate(hep_pcs.index.values):
    ax.annotate(txt, (0, 0), (hep_pcs.PC1.values[i], hep_pcs.PC2.values[i]),
            arrowprops=dict(arrowstyle='<-', color='black', linewidth=1.5),
            xycoords='data', textcoords='data', fontsize=12, color="black")

ax.set_title('PC1 (below average) vs. PC2 (javelin))')
ax.set_xlabel('principal component 1 (below average)')
ax.set_ylabel('principal component 2 (javelin)')
plt.show()

---

### 10) Plot PC1 versus score. Do our results make sense?

Remember: **Always interpret your results**. Because we claimed that PC1 described "below average" athletes, we would guess that a larger PC1 value would have a lower score.

Let's graph it below and see if the scores agree with our intuition.

In [ ]:
# A:

**Does this graph align with our guess from above?** If not, either our calculations or interpretation are likely incorrect. Thinking about the principal components allows us to provide evidence that our results are correct and helps us better understand the PCA results.

---

### 10.A) Plot PC2 versus score. What does this tell you about the relationship between the events and the score?

Consider how an athlete's score would be affected if they are better at javelin (our interpretation of PC2). After you understand how score _should be_ affected by PC2, look at the graph below. Does it align with your expectation?

In [ ]:
# A:

**Tip:** Notice that we have an outlier in the bottom-right corner. Outliers are either genuine or an indicator of bad data. It's typically useful to find the athlete who corresponds with each outlier to determine if the data are bad.

---

<a id="partii"></a>
# Part II: Wine Quality Data Set

Now it's your turn! Try repeating this analysis to investigate the wine quality data set. As much as possible, try to perform the analysis without looking at what we did above. Remember that most of what we did previously was manipulating Pandas data and plotting.


### 1) Load the wine quality data set.

In [ ]:
wine = pd.read_csv('./datasets/wine_quality.csv')

wine.head()

---

### 2) Subset the wine data to everything except the `red_wine` column.

In [ ]:
# A:

---

### 3) Examine the correlation between variables.

In [ ]:
# A:

---

### 4) Standardize the variables.

In [ ]:
# A:

---

### 5) Fit a PCA on the standardized data.

Create a new DataFrame with the principal components and the `red_wine` column added back in from the original data.

In [ ]:
# A:

---

### 6) Create a DataFrame with the principal components.


In [ ]:
# A:

---

### 7) Plot the explained variance (ratio) of the components.

In [ ]:
# A:

---

### 8) Print out the component weights with their corresponding variables for PC1, PC2, and PC3.

How would you label the components based on their weights?

In [ ]:
# A:

**Note:** It's interesting to research what causes these factors in wines. For example, is sulfur dioxide added to wine or is it a natural byproduct of its ingredients? What affects how much sulfur dioxide a winemaker might add to wine? 

By researching these questions, you can likely find a good explanation for these principal components — beyond a bland description of the weightings. You can also use these intuitions from research (e.g., how pH and residual sugar affect sulfur dioxide) to provide evidence of the principal components' validity.

---

### 9) Plot PC1 versus PC2.

- Use a regular scatterplot.
- Vary the alpha value to see better densities.

In [ ]:
# A:

---

### 10. Plot a Seaborn pair plot of PC1, PC2, and PC3 with `hue='red_wine'`.

Do any of the components differentiate red and white wine? If so, what does this tell you about the difference between red and white wine based on the component weights? Does each plot align with your expectations based on the components?

In [ ]:
# A:

---

<a id="partiii"></a>
# Part III: Horn's Parallel Analysis

You can determine the appropriate number of components to keep by using a bootstrapping procedure known as Horn's parallel analysis. This is the gold standard in determining which components aren't noise.

How to perform the parallel analysis (pseudocode):

    For n iterations:
        Create normally distributed random data that are the same shape as your data.
        Fit a PCA on the random data.
        Pull out the eigenvalues.
    Select a percentile of the eigenvalues as your threshold (0.5 = median, 0.95 = 95 percent confidence, etc.).
    Plot the random component eigenvalues at that percentile against your data's PCA eigenvalues.
    Components above the selected percentile are not noise; those that are under are.
   
Ultimately, we are comparing the PCA-explained variance of your data set against the PCA-explained variance of random Gaussian functions. If your PCA's explained variance is lower than that of random Gaussians, then we’ll assume these components represent noise.

--- 

### 1) Write a function to perform the parallel analysis.

In [ ]:
# Replace with the actual algorithm.
def horn_parallel_analysis(shape, iters=1000, percentile=95):
    return [0.0] * shape[1]

---

### 2) Run parallel analysis for the heptathlon data.

In [ ]:
# This should work automatically (for 95th percentile).
hep_pa = horn_parallel_analysis(X.shape, percentile=95)
hep_pa

### 3) Run parallel analysis for the wine data.

In [ ]:
# A:

---

### 4) Plot the wine eigenvalues (`.variance_explained_`) against the parallel analysis random eigenvalue cut-offs.

How many components are not noise, based on the chart?

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

ax.plot(range(1, X.shape[1]+1), pca.explained_variance_, lw=2, marker='o')
ax.plot(range(1, X.shape[1]+1), hep_pa, lw=2, color='darkred', marker='o')

ax.set_title("Horns parallel analysis on heptathlon principal components")
ax.set_xlabel("Principal Component")
ax.set_ylabel("Eigenvalue")

plt.legend(['Actual PCA Variance Explained', "Random Gaussian PCA Variance Explained"])
plt.show()

**Conclusion:** Note that the actual variance explained is lower than the variance explained of the random Gaussian variance after PC1. Hence, a reasonable set of principal components would be the first.

---

### 5) Plot the wine eigenvalues (`.variance_explained_`) against the parallel analysis random eigenvalue cut-offs.

In [ ]:
# A: